<a href="https://colab.research.google.com/github/VLNTNF/four-in-a-row-bot/blob/main/Puissance4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 10 12:19:07 2020

@author: Valentin
"""

# -*- coding: utf-8 -*-

import numpy as np
import copy
import math
from timeit import default_timer as timer

CRED = '\033[31m'
CEND = '\033[0m'
CBLUE = '\033[36m'

grilleDimX = 12
grilleDimY = 6

long_branche = 4

def Affichage(s):
    for i in range(grilleDimY):
        print("|",end=' ')
        for j in range(grilleDimX):
            if(s[i][j]==1):
                print(CBLUE+'0'+CEND,end=' ')
            elif s[i][j]==2:
                print(CRED+'0'+CEND,end=' ')
            else:
                print(" ",end=' ')
            print("|",end=' ')
        print()
    print("|",end=' ')
    for i in range(grilleDimX):
        print("_",end=" ")
        print("|",end=' ')
    print()
    print("|",end=' ')
    for i in range(1, grilleDimX+1):
        print(i%10,end=" ")
        print("|",end=' ')
    print()
    
def Utility(s):
    u = 0
    for i in range(0, grilleDimY):
        for j in range(0, grilleDimX):
            # test sur les colonnes
            try:
                if s[i][j] == s[i][j + 1] == 1:
                    u += 10
                if s[i][j] == s[i][j + 1] == s[i][j + 2] == 1:
                    u += 100
                if s[i][j] == s[i][j+1] == s[i][j+2] == s[i][j+3] == 1:
                    u += 10000

                if s[i][j] == s[i][j + 1] == 2:
                    u -= 10
                if s[i][j] == s[i][j + 1] == s[i][j + 2] == 2:
                    u -= 100
                if s[i][j] == s[i][j+1] == s[i][j+2] == s[i][j+3] == 2:
                    u -= 10000
            except IndexError:
                pass
            # test sur les lignes
            try:
                if s[i][j] == s[i + 1][j] == 1:
                    u += 10
                if s[i][j] == s[i + 1][j] == s[i + 2][j] == 1:
                    u += 100
                if s[i][j] == s[i+1][j] == s[i+2][j] == s[i+3][j] == 1:
                    u += 10000

                if s[i][j] == s[i + 1][j] == 2:
                    u -= 10
                if s[i][j] == s[i + 1][j] == s[i + 2][j] == 2:
                    u -= 100
                if s[i][j] == s[i+1][j] == s[i+2][j] == s[i+3][j] == 2:
                    u -= 10000
            except IndexError:
                pass
            # test sur les diagonales
            try:
                if not j + 3 > 6 and s[i][j] == s[i + 1][j + 1] == 1:
                    u += 100
                if not j + 3 > 6 and s[i][j] == s[i + 1][j + 1] == s[i + 2][j + 2] == 1:
                    u += 100
                if not j + 3 > 6 and s[i][j] == s[i+1][j + 1] == s[i+2][j + 2] == s[i+3][j + 3] == 1:
                    u += 10000

                if not j + 3 > 6 and s[i][j] == s[i + 1][j + 1] == 2:
                    u -= 100
                if not j + 3 > 6 and s[i][j] == s[i + 1][j + 1] == s[i + 2][j + 2] == 2:
                    u -= 100
                if not j + 3 > 6 and s[i][j] == s[i+1][j + 1] == s[i+2][j + 2] == s[i+3][j + 3] == 2:
                    u -= 10000
            except IndexError:
                pass
            try:
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == 1:
                    u += 10
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == 1:
                    u += 100
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == s[i+3][j - 3] == 1:
                    u += 10000

                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == 2:
                    u -= 10
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == 2:
                    u -= 100
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == s[i+3][j - 3] == 2:
                    u -= 10000
            except IndexError:
                pass
    return u

def TerminalTest(s):
    for i in range(0, grilleDimY):
        for j in range(0, grilleDimX):
            try:
                if s[i][j] == s[i+1][j] == s[i+2][j] == s[i+3][j] == 1:
                    return True
                if s[i][j] == s[i+1][j] == s[i+2][j] == s[i+3][j] == 2:
                    return True
            except IndexError:
                pass
            try:
                if s[i][j] == s[i][j+1] == s[i][j+2] == s[i][j+3] == 1:
                    return True
                if s[i][j] == s[i][j+1] == s[i][j+2] == s[i][j+3] == 2:
                    return True
            except IndexError:
                pass
            try:
                if not j + 3 > 6 and s[i][j] == s[i+1][j + 1] == s[i+2][j + 2] == s[i+3][j + 3] == 1:
                    return True
                if not j + 3 > 6 and s[i][j] == s[i+1][j + 1] == s[i+2][j + 2] == s[i+3][j + 3] == 2:
                    return True
            except IndexError:
                pass
            try:
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == s[i+3][j - 3] == 1:
                    return True
                if not j - 3 < 0 and s[i][j] == s[i+1][j - 1] == s[i+2][j - 2] == s[i+3][j - 3] == 2:
                    return True
            except IndexError:
                pass
    if np.count_nonzero(s[0])==grilleDimX:
        return True
    return False
        
def Actions(s):
    return np.where(s[0] == 0)[0]
          
def Result(s, p, a):
    for i in range(grilleDimY-1,-1,-1):
        if(s[i][a]==0):
            s[i][a]=p
            return s
        
def ResultTest(s, p, a):
    scopy = copy.deepcopy(s)
    for i in range(grilleDimY-1,-1,-1):
        if(scopy[i][a]==0):
            scopy[i][a]=p
            return scopy

def AlphaBetaSearching(s):
    v, ak = MaxValue(s, 0, -math.inf, math.inf)
    return ak
    
def MinValue(s, t, alpha, beta):
    if (TerminalTest(s) or (t >= long_branche)):
        return Utility(s)
    v = math.inf
    for a in Actions(s):
        v2, ak = MaxValue(ResultTest(s, 2, a), t+1, alpha, beta)
        v = min(v, v2)
        if (v <= alpha):
            return v
        beta = min(beta, v)
    return v

def MaxValue(s, t, alpha, beta):
    if (TerminalTest(s) or (t >= long_branche)):
        return Utility(s), -1
    v = -math.inf
    listv = [v]
    listActions = Actions(s)
    for a in listActions:
        listv.append(MinValue(ResultTest(s, 1, a), t+1, alpha, beta))
        v = max(listv)
        ab = listActions[listv.index(v)-1]
        if (v >= beta):
            return v, ab
        alpha = max(alpha,v)     
    return v, ab

def QuiCommence():
    while(True):
        rep = input("IA commence? oui/non: ")
        if((rep=="oui") or (rep=="non")):
            return (rep=="oui")
        
def Demande():
    print(CRED+"\nColonne entre 1 et "+ str(grilleDimX) +": "+CEND, end='')
    res = input()
    return (int(res) - 1)

def Jeu():
    s=np.zeros((grilleDimY,grilleDimX),dtype=np.byte)
    commence = QuiCommence()
    if(not commence):
        Result(s, 2, Demande())
        print()
        Affichage(s)  
    while(True):
        starttime = timer()
        if(commence):
            a = (grilleDimX//2)-1
            commence = False
        else:
            a = AlphaBetaSearching(s)
        print(CBLUE+"\n#################################################"+CEND)
        print(CBLUE+f"\nJoue en: {a+1} ({round(timer() - starttime, 2)}s)"+CEND)
        Result(s, 1, a)
        print()
        Affichage(s)
        if(TerminalTest(s)):
            break
        print(CRED+"\n#################################################"+CEND)
        Result(s, 2, Demande())
        print()
        Affichage(s)
        if(TerminalTest(s)):
            break
    print()
    if(np.count_nonzero(s[0])==grilleDimX):
        print("Match nul")
    elif(Utility(s) > 0):
        print(CBLUE+"IA gagne"+CEND)
    else:
        print(CRED+"Joueur gagne"+CEND)

Jeu()

IA commence? oui/non: oui

#################################################

Joue en: 6 (0.0s)

|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   | 0 |   |   |   |   |   |   | 
| _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | 
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 0 | 1 | 2 | 

#################################################

Colonne entre 1 et 12: 2

|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   |   |   |   |   |   |   |   |   |   |   |   | 
|   | 0 |   |   |   | 0 |   |   |   |   |   |   | 
| _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | 
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 0 | 1 | 2 | 

########